# Packages

In [ ]:
import plotly.express as px
import pandas as pd
from google.cloud import bigquery
import numpy as np
from datetime import time, timedelta, datetime
import math 
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ipywidgets
import statsmodels.stats.weightstats as sw
from statsmodels.stats.proportion import test_proportions_2indep
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_format='retina'
from google.cloud import bigquery
from scipy.stats import norm
from scipy import stats
pd.options.display.float_format = "{:,.2f}".format
from scipy.stats import norm
from scipy import stats
import statsmodels.stats.weightstats as ws
import openpyxl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import AB_library
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
# df["Week"] = df["Date"].dt.to_period("W").dt.to_timestamp()

bigquery_client = bigquery.Client(project='analytics-dev-333113')

def cycle_sql(start, end, query, weeks=False):
    """
    You have to use {date} in your script to add cycle date into this backets
    """
    date_start = datetime.strptime(start, '%Y-%m-%d')
    date_end = datetime.strptime(end, '%Y-%m-%d')

    if weeks == False:
        daterange = [(date_start + timedelta(days=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days)+1)]
    else:
        daterange = [(date_start + timedelta(weeks=x)).strftime('%Y-%m-%d') for x in range(((date_end-date_start).days//7)+1)] # weeks dividing days by 7

    total_df = pd.DataFrame()

    counter = 0

    for date in daterange:
        counter+=1
        print(f"{counter}) Uploading - {date}:", datetime.today().strftime('%Y-%m-%d %H:%M:%S'))
        script = query.format(date = date)
        df_cycle = bigquery_client.query(script).to_dataframe()
        if df_cycle.empty == True:
            print('Dataframe is empty')
        total_df = pd.concat([df_cycle, total_df])
    return total_df

def read_bq(query, project='analytics-dev-333113'):
    client = bigquery.Client(project=project)
    query_job = client.query(query)
    result_df = query_job.to_dataframe()
    return result_df

def display_side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(
        html_str.replace('table','table style="display:inline"'), 
        raw=True
    )

def writing_excel(name:str, dataset1=None, dataset2=None, dataset3=None, dataset4=None):
    with pd.ExcelWriter(f"{name}.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index 
    # to store the dataframe in specified sheet

        if dataset1 is not None:
            if dataset2 is not None:
                if dataset3 is not None:
                    if dataset4 is not None:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                        dataset4.to_excel(writer, sheet_name=f"4-{name}", 
                                        #   index=False
                                          )
                    else:
                        dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                        #   index=False
                                          )
                        dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                        #   index=False
                                          )
                        dataset3.to_excel(writer, sheet_name=f"3-{name}", 
                                        #   index=False
                                          )
                else:
                    dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                    #   index=False
                                      )
                    dataset2.to_excel(writer, sheet_name=f"2-{name}", 
                                    #   index=False
                                      )
            else:
                dataset1.to_excel(writer, sheet_name=f"1-{name}", 
                                #   index=False
                                  )

        print('DataFrame is written to Excel File successfully.')

# G-Force analysis

$$ \text{Acceleration [in g]} = \frac{\text{Acceleration [in м/с²]}}{9.8} $$


$$ \text{Acceleration [in м/s²]} = \text{Acceleration [in g]} \times 9.8 $$


In [15]:
import math

# --- КОНСТАНТЫ ---
# Ускорение свободного падения (g) в м/с²
G_CONSTANT = 9.81

# ##############################################################################
# === ПАРАМЕТРЫ ДЛЯ МОДЕЛИРОВАНИЯ (ЗДЕСЬ МОЖНО МЕНЯТЬ ЗНАЧЕНИЯ) ===
# ##############################################################################

# --- Общие параметры автомобиля ---
# Скорость, с которой автомобиль движется перед торможением или поворотом.
CAR_SPEED_KMH = 50.0  # в км/ч

# --- Параметры для сценария "Ускорение" ---
# За сколько секунд автомобиль разгоняется с 0 до CAR_SPEED_KMH.
ACCELERATION_TIME_S = 3.0  # в секундах

# --- Параметры для сценария "Остановки" ---
# За сколько секунд автомобиль полностью останавливается со скорости CAR_SPEED_KMH.
# Для имитации резкого/аварийного торможения используйте малое значение (например, 1.5 - 2.5 с).
BRAKING_TIME_S = 1.0  # в секундах

# --- Параметры для сценария "Повороты" ---
# Радиус поворота в метрах. Чем МЕНЬШЕ радиус, тем РЕЗЧЕ поворот.
# > 50 м: плавный поворот на шоссе
# 15-30 м: обычный городской поворот
# < 10 м: очень резкий, агрессивный поворот
TURN_RADIUS_M = 21.0  # в метрах

# --- Параметры для сценария "Манипуляции с девайсом" (Падение) ---
# Высота, с которой падает телефон.
DROP_HEIGHT_M = 1  # в метрах (например, с уровня рук)

# Предполагаемое время удара о поверхность. Это время, за которое скорость падает до 0.
# Для твердых поверхностей (асфальт, плитка) оно очень мало.
IMPACT_DURATION_S = 0.025 # в секундах

# ##############################################################################
# === ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ (ИХ МЕНЯТЬ НЕ НУЖНО) ===
# ##############################################################################

def kmh_to_ms(speed_kmh: float) -> float:
    """Переводит скорость из км/ч в м/с."""
    return speed_kmh * 1000 / 3600

def ms2_to_g(acceleration_ms2: float) -> float:
    """Переводит ускорение из м/с² в g."""
    return acceleration_ms2 / G_CONSTANT

def calculate_total_g_force(dynamic_g: float) -> float:
    """
    Рассчитывает суммарный вектор ускорения, который зафиксирует акселерометр,
    складывая динамическое ускорение с вектором гравитации (1g).
    """
    return math.sqrt(1**2 + dynamic_g**2)

# ##############################################################################
# === ОСНОВНАЯ ФУНКЦИЯ ДЛЯ РАСЧЕТА И ВЫВОДА РЕЗУЛЬТАТОВ ===
# ##############################################################################

def run_all_scenarios():
    """
    Выполняет расчеты для всех сценариев на основе заданных параметров
    и выводит результаты в консоль.
    """
    print("="*50)
    print("Результаты моделирования для акселерометра")
    print("="*50 + "\n")

    # --- Сценарий 1: Ускорение ---
    print(f"--- 1. Сценарий: УСКОРЕНИЕ ---")
    print(f"Расчет для разгона с 0 до {CAR_SPEED_KMH} км/ч за {ACCELERATION_TIME_S} с.")
    
    start_v_ms = 0
    end_v_ms = kmh_to_ms(CAR_SPEED_KMH)
    accel_ms2 = abs(end_v_ms - start_v_ms) / ACCELERATION_TIME_S
    dynamic_g = ms2_to_g(accel_ms2)
    total_g = calculate_total_g_force(dynamic_g)
    
    print(f"  -> Чистое ускорение (от движения): {dynamic_g:.2f} g")
    print(f"  -> Итоговый показатель на датчике: {total_g:.2f} g\n")

    # --- Сценарий 2: Остановки (Резкое торможение) ---
    print(f"--- 2. Сценарий: ОСТАНОВКИ ---")
    print(f"Расчет для торможения с {CAR_SPEED_KMH} км/ч до 0 за {BRAKING_TIME_S} с.")
    
    start_v_ms = kmh_to_ms(CAR_SPEED_KMH)
    end_v_ms = 0
    brake_ms2 = abs(end_v_ms - start_v_ms) / BRAKING_TIME_S
    dynamic_g = ms2_to_g(brake_ms2)
    total_g = calculate_total_g_force(dynamic_g)
    
    print(f"  -> Чистое замедление (от торможения): {dynamic_g:.2f} g")
    print(f"  -> Итоговый показатель на датчике: {total_g:.2f} g\n")

    # --- Сценарий 3: Повороты ---
    print(f"--- 3. Сценарий: ПОВОРОТЫ ---")
    print(f"Расчет для поворота радиусом {TURN_RADIUS_M} м на скорости {CAR_SPEED_KMH} км/ч.")
    
    speed_ms = kmh_to_ms(CAR_SPEED_KMH)
    turn_ms2 = (speed_ms ** 2) / TURN_RADIUS_M
    dynamic_g = ms2_to_g(turn_ms2)
    total_g = calculate_total_g_force(dynamic_g)
    
    print(f"  -> Боковое ускорение (от поворота): {dynamic_g:.2f} g")
    print(f"  -> Итоговый показатель на датчике: {total_g:.2f} g\n")
    
    # --- Сценарий 4: Манипуляции с девайсом (Падение) ---
    print(f"--- 4. Сценарий: МАНИПУЛЯЦИИ С ДЕВАЙСОМ (Падение) ---")
    print(f"Расчет для падения телефона с высоты {DROP_HEIGHT_M} м.")

    # Фаза 1: Свободное падение
    freefall_time = math.sqrt((2 * DROP_HEIGHT_M) / G_CONSTANT)
    
    # Фаза 2: Удар
    impact_velocity_ms = math.sqrt(2 * G_CONSTANT * DROP_HEIGHT_M)
    impact_accel_ms2 = impact_velocity_ms / IMPACT_DURATION_S
    impact_g = ms2_to_g(impact_accel_ms2)
    
    print(f"  -> Фаза 1 (полет): Состояние невесомости (~0.0 g) в течение {freefall_time:.2f} с.")
    print(f"  -> Фаза 2 (удар): Пиковое ускорение составит примерно {impact_g:.2f} g\n")
    
    print("="*50)


run_all_scenarios()


Результаты моделирования для акселерометра

--- 1. Сценарий: УСКОРЕНИЕ ---
Расчет для разгона с 0 до 50.0 км/ч за 3.0 с.
  -> Чистое ускорение (от движения): 0.47 g
  -> Итоговый показатель на датчике: 1.11 g

--- 2. Сценарий: ОСТАНОВКИ ---
Расчет для торможения с 50.0 км/ч до 0 за 1.0 с.
  -> Чистое замедление (от торможения): 1.42 g
  -> Итоговый показатель на датчике: 1.73 g

--- 3. Сценарий: ПОВОРОТЫ ---
Расчет для поворота радиусом 21.0 м на скорости 50.0 км/ч.
  -> Боковое ускорение (от поворота): 0.94 g
  -> Итоговый показатель на датчике: 1.37 g

--- 4. Сценарий: МАНИПУЛЯЦИИ С ДЕВАЙСОМ (Падение) ---
Расчет для падения телефона с высоты 1 м.
  -> Фаза 1 (полет): Состояние невесомости (~0.0 g) в течение 0.45 с.
  -> Фаза 2 (удар): Пиковое ускорение составит примерно 18.06 g

